# OSNAP Parcels Experiments


## Technical preamble

In [ ]:
# import matplotlib.colors as colors
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import xarray as xr
from datetime import datetime, timedelta
# import seaborn as sns
# from matplotlib.colors import ListedColormap
import cmocean as co
import pandas as pd
import matplotlib.dates as mdates


In [ ]:
## Parameters
# Project path
project_path = Path.cwd() / '..' / '..' 
project_path = project_path.resolve()

# Parcels track data file
path_data_tracks = Path('data/processed/tracks/osnap/') 
filename_tracks = 'tracks_osnap_forwards_201701_N19886_D365_Rnd123.nc'

# model mask file
data_path = Path("data/external/iAtlantic/")
experiment_name = "VIKING20X.L46-KKG36107B"
mesh_mask_file = project_path / data_path / "mask" / experiment_name / "1_mesh_mask.nc"

#section lonlat file
sectionPath = Path('data/external/sections/')
sectionFilename = 'osnap_pos_wp.txt'
sectionname = 'osnap'
gsrsectionFilename = 'gsr_pos_wp.txt'

degree2km = 1.852*60.0

# choose color map, suggest amp for backwards, tempo_r for forwards

cmap = co.cm.tempo_r
# cmap = co.cm.amp


## plotting functions

In [ ]:
# range of stations from west to east, stations 0-12. Python indexing.

def plotTracksBetweenStations(ds,stationRange,xlims,ylims,cmap=co.cm.tempo_r):
    
    dsmask = ds.where(ds.isel(obs=0).lon > lonlat.lon[stationRange[0]]).where(ds.isel(obs=0).lon < lonlat.lon[stationRange[1]])

    plt.figure(figsize = (16,8))

    # plt.scatter(
    #     ds.where(ds.alive == 1).lon.data.flatten(),
    #     ds.where(ds.alive == 1).lat.data.flatten(),
    #     3,
    #     ds.where(ds.alive == 1).temp.data.flatten(),
    #     alpha=0.3, vmin = 2, vmax = 7
    # )
    plt.scatter(
        dsmask.lon.data.flatten(),
        dsmask.lat.data.flatten(),
        3,
#         (dsmask.time.data.flatten()-ds.time.isel(obs=0,traj=0).data).astype('timedelta64[D]')
        mdates.date2num(dsmask.time.data.flatten())
#         dsmask.temp.data.flatten()
        ,cmap= cmap
        ,zorder=2
    #   ,alpha=0.3
    )
    cb = plt.colorbar(label = "date");
    loc = mdates.MonthLocator()
    cb.ax.yaxis.set_major_locator(loc)
    cb.ax.yaxis.set_major_formatter(mdates.ConciseDateFormatter(loc))

    # plt.plot(titanic_lon, titanic_lat, 'rx', ms = 8, mew = 2);

    depth.isel(y=slice(1400, 2499), x=slice(0, 2404)).plot.contour(
        x="nav_lon", y="nav_lat", colors = 'grey', levels = [200,800,1500,2000,2500,3500],zorder=1
    );
    depth.isel(y=slice(1400, 2499), x=slice(0, 2404)).plot.contour(
        x="nav_lon", y="nav_lat", colors = 'k', levels = [1],zorder=3
    );

    plt.scatter(
        ds.lon.isel(obs=0).data.flatten(),
        ds.lat.isel(obs=0).data.flatten(),2,zorder=4
    )
    plt.scatter(
        dsmask.lon.isel(obs=0).data.flatten(),
        dsmask.lat.isel(obs=0).data.flatten(),2,zorder=5
    )

    # plt.savefig('upstream.png')
    plt.ylim(ylims[0],ylims[1])
    plt.xlim(xlims[0],xlims[1])
    # plt.ylim(62,65)
    # plt.xlim(-10,-2.5)


## Load data

In [ ]:
ds = xr.open_dataset(project_path / path_data_tracks / filename_tracks)

display(ds)
# ds.isel(obs=0).z.max()

In [ ]:
ds.isel(obs=0).time


In [ ]:
mesh_mask = xr.open_dataset(mesh_mask_file)
mesh_mask = mesh_mask.squeeze()
mesh_mask = mesh_mask.set_coords(["nav_lon", "nav_lat", "nav_lev"])

bathy = mesh_mask.mbathy.rename("number of water filled points")

depth = (mesh_mask.e3t_0 * mesh_mask.tmask).sum("z")
# display(mesh_mask)

### section position data

In [ ]:
lonlat = xr.Dataset(pd.read_csv(project_path / sectionPath / sectionFilename,delim_whitespace=True))
lonlat['lon'] *= -1.0

lonlat

In [ ]:
lonlat.lon.attrs['long_name']='Longitude'
lonlat.lat.attrs['long_name']='Latitude'
lonlat.lon.attrs['standard_name']='longitude'
lonlat.lat.attrs['standard_name']='latitude'
lonlat.lon.attrs['units']='degrees_east'
lonlat.lat.attrs['units']='degrees_north'

lonlat2mean= lonlat.rolling({'dim_0':2}).mean()

lonlatdiff = abs(lonlat.diff('dim_0'))
lonlatdiff['lat'] = lonlatdiff['lat']*degree2km
lonlatdiff['lon'] = lonlatdiff['lon']*degree2km*np.cos(np.radians(lonlat2mean.lat.data[1:]))
lonlatdiff=lonlatdiff.assign({'length':np.sqrt(lonlatdiff['lon']**2+lonlatdiff['lat']**2)})

total_length = lonlatdiff.length.sum().data

In [ ]:
lonlat.plot.scatter(x='lon',y='lat')
lonlat2mean.plot.scatter(x='lon',y='lat')

## Have a quick look

### Release positions of the particles

In [ ]:
plt.figure(figsize = (9,4))
plt.scatter(ds.isel(obs=0).lon.data.flatten(),ds.isel(obs=0).z.data.flatten(),s = 2)
plt.ylim(4000,0)

### Trajectories of all particles

In [ ]:
plotTracksBetweenStations(ds,stationRange=[0,12],xlims=[-75,15],ylims=[36,70],cmap=cmap)

## plot subsections

In [ ]:
plotTracksBetweenStations(ds,stationRange=[0,1],xlims=[-75,15],ylims=[36,70],cmap=cmap)

In [ ]:
plotTracksBetweenStations(ds,stationRange=[1,2],xlims=[-75,15],ylims=[36,70],cmap=cmap)

In [ ]:
plotTracksBetweenStations(ds,stationRange=[2,3],xlims=[-75,15],ylims=[36,70],cmap=cmap)

In [ ]:
plotTracksBetweenStations(ds,stationRange=[3,4],xlims=[-75,15],ylims=[36,70],cmap=cmap)

In [ ]:
plotTracksBetweenStations(ds,stationRange=[4,5],xlims=[-75,15],ylims=[36,70],cmap=cmap)

In [ ]:
plotTracksBetweenStations(ds,stationRange=[5,6],xlims=[-75,15],ylims=[36,70],cmap=cmap)

In [ ]:
plotTracksBetweenStations(ds,stationRange=[6,7],xlims=[-75,15],ylims=[36,70],cmap=cmap)

In [ ]:
plotTracksBetweenStations(ds,stationRange=[7,8],xlims=[-75,15],ylims=[36,70],cmap=cmap)

In [ ]:
plotTracksBetweenStations(ds,stationRange=[8,9],xlims=[-75,15],ylims=[36,70],cmap=cmap)

In [ ]:
plotTracksBetweenStations(ds,stationRange=[9,10],xlims=[-75,15],ylims=[36,70],cmap=cmap)

In [ ]:
plotTracksBetweenStations(ds,stationRange=[10,11],xlims=[-75,15],ylims=[36,70],cmap=cmap)

In [ ]:
# ds1 = ds.where(ds.time > np.datetime64('2016-07-16'))
plotTracksBetweenStations(ds,stationRange=[11,12],xlims=[-75,15],ylims=[36,70],cmap=cmap)

In [ ]:
# conda list